<a href="https://colab.research.google.com/github/magnetbrains-bit/Caterpillar-Tech-Challenge-2025/blob/main/CATERPILLAR_AI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#@title 1. Setup Environment, Clone Repository & Install Streaming Libs
import os
import sys
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='torchvision')
warnings.filterwarnings("ignore", category=FutureWarning)

NGROK_AUTH_TOKEN = "2wCr4ur5cPXoyb0P92RPsXHGdMM_7q6AMsVz2sYZHyp67YpBK" # <<< REPLACE THIS
print("Ngrok Authtoken variable set (ensure it's your valid token).")

print("\nInstalling libraries (including ultralytics for YOLO)...")
!pip install -q opencv-python-headless torch torchvision torchaudio ultralytics websockets pyngrok nest_asyncio matplotlib numpy xformers
print("Library installation complete.")

GIT_REPO_URL = "https://github.com/computervisionpro/Python-Depth-Est-AV2.git"
REPO_NAME = GIT_REPO_URL.split('/')[-1].replace('.git', '')

if not os.path.exists(REPO_NAME):
    print(f"\nCloning repository: {GIT_REPO_URL}")
    !git clone {GIT_REPO_URL}
else:
    print(f"\nRepository '{REPO_NAME}' already exists. Skipping clone.")

target_dir = f"/content/{REPO_NAME}"
if os.path.isdir(target_dir):
    if os.getcwd() != target_dir:
        try:
            %cd {target_dir}
            print(f"\nChanged directory to: {os.getcwd()}")
        except Exception as e:
            print(f"Error changing directory: {e}. Current dir: {os.getcwd()}")
    else:
        print(f"\nAlready in directory: {os.getcwd()}")
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
        print(f"Added '{os.getcwd()}' to sys.path")
else:
    print(f"!!! ERROR: Target directory '{target_dir}' does not exist. Cloning might have failed.")

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE" and NGROK_AUTH_TOKEN.strip() != "":
    print(f"\nConfiguring ngrok with the provided token...")
    try:
        !ngrok config add-authtoken {NGROK_AUTH_TOKEN}
        print("Ngrok configuration command executed.")
    except Exception as ngrok_err:
        print(f"ERROR during ngrok configuration: {ngrok_err}")
else:
    print("\nSkipping ngrok configuration due to missing or placeholder token.")

print("\n✅ Cell 1: Setup, Cloning, and Libs Installation Complete.")
print("-" * 70)

Ngrok Authtoken variable set (ensure it's your valid token).

Installing libraries (including ultralytics for YOLO)...
Library installation complete.

Repository 'Python-Depth-Est-AV2' already exists. Skipping clone.

Already in directory: /content/Python-Depth-Est-AV2

Configuring ngrok with the provided token...
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Ngrok configuration command executed.

✅ Cell 1: Setup, Cloning, and Libs Installation Complete.
----------------------------------------------------------------------


In [23]:
#@title 2. Download Depth Model Checkpoint and Load Models (Depth + YOLO)
import torch
import torch.nn as nn # Though nn might not be directly used here, often good to have with torch
import numpy as np
import os
import sys
from ultralytics import YOLO

# Ensure global model placeholders for other cells
_depth_model = None
_yolo_model = None
MODEL_TYPE = "vits" # As per your OCR examples
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model_configs = {} # Will be populated

# Ensure Repo Path if Cell 1's %cd didn't persist for python kernel
if 'REPO_NAME' not in globals(): REPO_NAME = "Python-Depth-Est-AV2" # Fallback
repo_path_check = f"/content/{REPO_NAME}"
if os.path.exists(repo_path_check) and repo_path_check not in sys.path:
    sys.path.append(repo_path_check)
    print(f"[Cell 2] Added {repo_path_check} to sys.path")
if os.path.isdir(repo_path_check) and os.getcwd() != repo_path_check:
    try:
        os.chdir(repo_path_check)
        print(f"[Cell 2] Changed directory to {os.getcwd()}")
    except Exception as e_chdir:
        print(f"[Cell 2 Warning] Could not change dir: {e_chdir}")

# === Depth Model Configuration and Loading (from your OCR structure) ===
CHECKPOINT_DIR = "checkpoints"
CHECKPOINT_NAME = f"depth_anything_v2_{MODEL_TYPE}.pth"
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, CHECKPOINT_NAME)
print(f"Depth Model Type: {MODEL_TYPE}")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

if MODEL_TYPE == 'vits': hf_model_name = 'Small'
elif MODEL_TYPE == 'vitb': hf_model_name = 'Base'
elif MODEL_TYPE == 'vitl': hf_model_name = 'Large'
elif MODEL_TYPE == 'vitg': hf_model_name = 'Giant'
else: hf_model_name = MODEL_TYPE.upper()
DOWNLOAD_URL = f"https://huggingface.co/depth-anything/Depth-Anything-V2-{hf_model_name}/resolve/main/{CHECKPOINT_NAME}"

if not os.path.exists(CHECKPOINT_PATH) or os.path.getsize(CHECKPOINT_PATH) < 1000:
    print(f"Downloading depth checkpoint: {DOWNLOAD_URL}...")
    download_status = os.system(f'wget -O "{CHECKPOINT_PATH}" "{DOWNLOAD_URL}"')
    if download_status != 0 or not (os.path.exists(CHECKPOINT_PATH) and os.path.getsize(CHECKPOINT_PATH) > 1000):
        print(f"!!! ERROR: Failed download checkpoint. wget status: {download_status}")
    else:
        print("Download complete.")
else:
    print(f"Depth Checkpoint '{CHECKPOINT_PATH}' already exists.")

model_configs = { # Made global for other cells if they need it
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}
print(f"Using device: {DEVICE} for PyTorch model loading.")

depth_model_loader = None # Local var for loading
if os.path.exists(CHECKPOINT_PATH) and os.path.getsize(CHECKPOINT_PATH) > 0:
    try:
        from depth_anything_v2.dpt import DepthAnythingV2
        print("Imported DepthAnythingV2.")
        if MODEL_TYPE not in model_configs:
            raise ValueError(f"Unsupported MODEL_TYPE '{MODEL_TYPE}'")

        config = model_configs[MODEL_TYPE]
        depth_model_loader = DepthAnythingV2(
            encoder=config['encoder'],
            features=config['features'],
            out_channels=config['out_channels']
        )
        # Debug after init
        if hasattr(depth_model_loader, 'img_size'):
            print(f"  DepthAnythingV2.img_size after init: {depth_model_loader.img_size}")
        else: print("  DepthAnythingV2 has no img_size attribute after init!")

        state_dict = torch.load(CHECKPOINT_PATH, map_location='cpu')
        depth_model_loader.load_state_dict(state_dict, strict=False)
        print("Depth model weights loaded.")
        depth_model_loader = depth_model_loader.to(DEVICE).eval()
        print(f"Depth model moved to {DEVICE} and set to eval mode.")
    except Exception as e_depth:
        print(f"!!! ERROR loading depth model: {e_depth}")
        traceback.print_exc()
        depth_model_loader = None
else:
    print("Depth checkpoint missing/invalid.")

_depth_model = depth_model_loader # Assign to global

# === YOLO Model Loading (from your OCR structure) ===
YOLO_MODEL_NAME = 'yolov8s.pt'
print(f"\nLoading YOLO model: {YOLO_MODEL_NAME}...")
yolo_model_loader = None # Local var
try:
    yolo_model_loader = YOLO(YOLO_MODEL_NAME)
    print(f"YOLO model '{YOLO_MODEL_NAME}' loaded.")
except Exception as e_yolo:
    print(f"!!! ERROR loading YOLO model: {e_yolo}")
_yolo_model = yolo_model_loader # Assign to global

print("-" * 70)
if _depth_model: print(f"✅ Depth Model '{MODEL_TYPE}' Ready.")
else: print(f"❌ Depth Model Loading FAILED.")
if _yolo_model: print(f"✅ YOLO Model '{YOLO_MODEL_NAME}' Ready.")
else: print(f"❌ YOLO Model Loading FAILED.")
print("Global vars _depth_model, _yolo_model, MODEL_TYPE, DEVICE, model_configs should be set.")
print("-" * 70)

Depth Model Type: vits
Depth Checkpoint 'checkpoints/depth_anything_v2_vits.pth' already exists.
Using device: cuda for PyTorch model loading.
Imported DepthAnythingV2.
  DepthAnythingV2 has no img_size attribute after init!
Depth model weights loaded.
Depth model moved to cuda and set to eval mode.

Loading YOLO model: yolov8s.pt...
YOLO model 'yolov8s.pt' loaded.
----------------------------------------------------------------------
✅ Depth Model 'vits' Ready.
✅ YOLO Model 'yolov8s.pt' Ready.
Global vars _depth_model, _yolo_model, MODEL_TYPE, DEVICE, model_configs should be set.
----------------------------------------------------------------------


In [24]:
#@title 3. Helper Function for Colorizing Depth Map (Original Structure)
import cv2
import numpy as np

COLORMAP_VIS = cv2.COLORMAP_INFERNO # As per PDF Cell 6 config in OCR

def colorize_relative_depth(raw_depth_map, colormap=COLORMAP_VIS): # This name matches your PDF's Cell 3
    """Applies a colormap to a raw depth map for visualization."""
    if raw_depth_map is None:
        # print("[Warn] colorize_relative_depth received None input.") # Original didn't print
        return None # Return None if input is None
    if not isinstance(raw_depth_map, np.ndarray):
        # print(f"[Warn] colorize_relative_depth received non-numpy input: {type(raw_depth_map)}") # Original didn't print
        return None

    vis_map = raw_depth_map.astype(np.float32)
    min_val = np.min(vis_map)
    max_val = np.max(vis_map)

    if max_val > min_val:
        normalized = (vis_map - min_val) / (max_val - min_val)
        depth_uint8 = (normalized * 255).astype(np.uint8)
    else:
        # print("[Warn] Depth map has zero range (min == max). Returning black image.") # Original didn't print
        depth_uint8 = np.zeros_like(vis_map, dtype=np.uint8)

    try:
        colored_depth = cv2.applyColorMap(depth_uint8, colormap)
        return colored_depth
    except Exception as e:
        # print(f"[Error] Failed to apply colormap: {e}") # Original didn't print
        return None # Return None if colormapping fails (as per OCR page 9 logic)

print("✅ Cell 3: Helper function 'colorize_relative_depth' (original structure) defined.")
print("-" * 70)

✅ Cell 3: Helper function 'colorize_relative_depth' (original structure) defined.
----------------------------------------------------------------------


In [29]:
#@title 4. Upload Video File for Streaming
from google.colab import files # Specific to Google Colab
import os
# import time # time not used in OCR version of this cell

# --- Check if running in Colab, otherwise skip file upload ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

input_video_path = None

if IN_COLAB:
    print("Please upload the video file you want to process (.mp4, .avi, etc.):")
    uploaded_video = files.upload()

    if uploaded_video:
        filename = list(uploaded_video.keys())[0]
        save_dir = "/content/"
        destination_path = os.path.join(save_dir, filename)
        try:
            with open(destination_path, 'wb') as f:
                f.write(uploaded_video[filename])
            print(f"\nUploaded '{filename}' ({len(uploaded_video[filename])} bytes)")
            print(f"Saved to: '{destination_path}'")
            input_video_path = destination_path
        except Exception as e:
            print(f"\nError saving uploaded file: {e}")
            input_video_path = None
    else:
        print("\nNo video file was uploaded.")
        input_video_path = None # Explicitly None
else:
    print("Not running in Google Colab. Skipping interactive upload.")
    print("Please ensure 'input_video_path' is set manually if not using Colab upload.")
    # Example: input_video_path = "my_local_video.mp4"


if input_video_path and os.path.exists(input_video_path):
    print(f"\n✅ Cell 4: Video path set to '{input_video_path}'.")
elif IN_COLAB and not input_video_path : # Only relevant if in Colab and upload failed
     print(f"\n❌ Cell 4: Video path NOT set in Colab. Upload may have failed or was skipped.")
else: # Covers not IN_COLAB and input_video_path not set manually
    print(f"\n⚠️ Cell 4: Video path not set. If running locally, please set 'input_video_path' manually.")
print("-" * 70)

Please upload the video file you want to process (.mp4, .avi, etc.):


Saving Generate_Crowded_Street_Scene_Video.mp4 to Generate_Crowded_Street_Scene_Video.mp4

Uploaded 'Generate_Crowded_Street_Scene_Video.mp4' (6560482 bytes)
Saved to: '/content/Generate_Crowded_Street_Scene_Video.mp4'

✅ Cell 4: Video path set to '/content/Generate_Crowded_Street_Scene_Video.mp4'.
----------------------------------------------------------------------


In [30]:
#@title 5. CORRECTED - Helper Functions & Configs (BEV Constants Scope for Dots & Trails)
import cv2
import numpy as np
import logging
from collections import deque # For object trails

# --- Configuration ---
COLOR_TEXT_BOX = (255, 255, 255)

# --- BEV Configuration - DEFINED HERE TO BE IN SCOPE FOR create_enhanced_bev ---
BEV_HEIGHT = 240; BEV_WIDTH = 320
BEV_BACKGROUND_COLOR = (30, 30, 30) # Darker Background
EGO_VEHICLE_COLOR_BEV = (0, 220, 0); EGO_VEHICLE_RECT_W = 20; EGO_VEHICLE_RECT_H = 12
VEHICLE_POSITION_BEV = (BEV_WIDTH // 2, BEV_HEIGHT - 20) # Ego slightly higher

# Dot Radii based on Danger Level
BEV_DOT_RADIUS_DANGER = 8
BEV_DOT_RADIUS_CAUTION = 6
BEV_DOT_RADIUS_FAR = 4
BEV_DOT_RADIUS_MIN = 2 # Explicitly define MIN for trails, must be >=1

BEV_Y_SCALE_RAW_FRACTION = BEV_HEIGHT * 0.90 # Use more of the BEV height
BEV_X_SCALE_FACTOR = BEV_WIDTH * 0.8
BEV_DOT_BORDER_COLOR = (200, 200, 200); BEV_DOT_BORDER_THICKNESS = 1

# Blinking Red Dots
BLINK_FRAME_INTERVAL = 10

# Object Trails
MAX_TRAIL_LENGTH = 5
BEV_OBJECT_TRACKS = {} # This will be reset in Cell 6's run_websocket_server
NEXT_TRACK_ID = 0    # This will be reset in Cell 6's run_websocket_server
TRACKING_MAX_DIST = 35 # Increased slightly

# --- Constants for determining dot/box colors (mirroring PDF logic from Cell 6) ---
PDF_COLOR_CLOSE = (0, 0, 255)    # Red
PDF_COLOR_MEDIUM = (0, 255, 255) # Yellow
PDF_COLOR_FAR = (0, 255, 0)      # Green

PDF_RAW_VAL_CLOSE_THRESHOLD = 0.6 # Must match Cell 6 if PDF lines use it directly from there
PDF_RAW_VAL_MEDIUM_THRESHOLD = 0.3

# --- Other Constants for Enhancements ---
ENHANCED_CONFIDENCE_THRESHOLD = 0.4
ENHANCED_TARGET_CLASSES = []


# get_raw_depth_at_center_for_enhancements (Same as before)
def get_raw_depth_at_center_for_enhancements(raw_depth_map, box_coords, object_class_name="Obj"):
    if raw_depth_map is None: logging.debug(f"Obj: {object_class_name} - Raw depth map None."); return None
    x1,y1,x2,y2=map(int,box_coords); cx,cy=(x1+x2)/2,(y1+y2)/2
    try:
        h,w=raw_depth_map.shape[:2]; sy,sx=max(0,min(h-1,int(round(cy)))),max(0,min(w-1,int(round(cx))))
        val=raw_depth_map[sy,sx]
        if not np.isfinite(val): logging.debug(f"Obj: {object_class_name} - Depth not finite: {val}"); return None
        return val
    except Exception as e: logging.error(f"Obj: {object_class_name} - Get depth error: {e}"); return None

# draw_ar_bounding_boxes (Same as before - uses PDF_COLOR_* defined above in this cell)
def draw_ar_bounding_boxes(frame_to_draw_on, yolo_results,
                           raw_depth_map_for_color, current_frame_min_raw, current_frame_max_raw,
                           yolo_model_names):
    if yolo_results is None or len(yolo_results) == 0 or yolo_results[0].boxes is None: return frame_to_draw_on
    boxes_data = yolo_results[0].boxes
    for i in range(len(boxes_data.xyxy)):
        box=boxes_data.xyxy[i].cpu().numpy(); conf=boxes_data.conf[i].cpu().numpy()
        cls_id=int(boxes_data.cls[i].cpu().numpy()); class_name=yolo_model_names[cls_id]
        if ENHANCED_TARGET_CLASSES and class_name not in ENHANCED_TARGET_CLASSES: continue
        if conf < ENHANCED_CONFIDENCE_THRESHOLD: continue
        x1,y1,x2,y2=map(int,box); obj_raw_depth=None
        if raw_depth_map_for_color is not None: obj_raw_depth=get_raw_depth_at_center_for_enhancements(raw_depth_map_for_color,box,class_name)
        box_color=PDF_COLOR_FAR
        depth_text="(Depth N/A)"
        if obj_raw_depth is not None and (current_frame_max_raw > current_frame_min_raw):
            depth_text=f"({obj_raw_depth:.2e})"
            d_range=current_frame_max_raw-current_frame_min_raw
            close_thr=current_frame_min_raw+d_range*PDF_RAW_VAL_CLOSE_THRESHOLD
            med_thr=current_frame_min_raw+d_range*PDF_RAW_VAL_MEDIUM_THRESHOLD
            if obj_raw_depth>=close_thr: box_color=PDF_COLOR_CLOSE
            elif obj_raw_depth>=med_thr: box_color=PDF_COLOR_MEDIUM
        txt=f"{class_name}({conf:.2f}){depth_text}"; cv2.rectangle(frame_to_draw_on,(x1,y1),(x2,y2),box_color,2)
        (tw,th),_=cv2.getTextSize(txt,cv2.FONT_HERSHEY_SIMPLEX,0.5,1)
        tbg_y1=max(0,y1-th-4); ty_pos=max(th,y1-4)
        cv2.rectangle(frame_to_draw_on,(x1,tbg_y1),(x1+tw,y1),box_color,-1)
        cv2.putText(frame_to_draw_on,txt,(x1,ty_pos),cv2.FONT_HERSHEY_SIMPLEX,0.5,COLOR_TEXT_BOX,1,cv2.LINE_AA)
    return frame_to_draw_on

def update_bev_tracks(current_detections_bev): # Same as before
    global BEV_OBJECT_TRACKS, NEXT_TRACK_ID
    if not BEV_OBJECT_TRACKS:
        for det in current_detections_bev:
            BEV_OBJECT_TRACKS[NEXT_TRACK_ID] = deque([ (det['x_base'], det['y_base'], det['color']) ], maxlen=MAX_TRAIL_LENGTH)
            det['track_id'] = NEXT_TRACK_ID; NEXT_TRACK_ID += 1
        return current_detections_bev
    unmatched_track_ids = list(BEV_OBJECT_TRACKS.keys()); unmatched_detections = list(range(len(current_detections_bev)))
    for track_id in list(unmatched_track_ids):
        last_x, last_y, _ = BEV_OBJECT_TRACKS[track_id][-1]; best_dist = float('inf'); best_det_idx = -1
        for det_idx in list(unmatched_detections):
            det = current_detections_bev[det_idx]; dist = np.sqrt((last_x - det['x_base'])**2 + (last_y - det['y_base'])**2)
            if dist < TRACKING_MAX_DIST and dist < best_dist: best_dist = dist; best_det_idx = det_idx
        if best_det_idx != -1:
            matched_det = current_detections_bev[best_det_idx]
            BEV_OBJECT_TRACKS[track_id].append( (matched_det['x_base'], matched_det['y_base'], matched_det['color']) )
            matched_det['track_id'] = track_id; unmatched_track_ids.remove(track_id); unmatched_detections.remove(best_det_idx)
    for track_id in unmatched_track_ids: del BEV_OBJECT_TRACKS[track_id]
    for det_idx in unmatched_detections:
        det = current_detections_bev[det_idx]
        BEV_OBJECT_TRACKS[NEXT_TRACK_ID] = deque([ (det['x_base'], det['y_base'], det['color']) ], maxlen=MAX_TRAIL_LENGTH)
        det['track_id'] = NEXT_TRACK_ID; NEXT_TRACK_ID += 1
    return current_detections_bev

def create_enhanced_bev(yolo_results,
                        raw_depth_map_for_color, current_frame_min_raw, current_frame_max_raw,
                        yolo_model_names, original_frame_width, current_frame_number):
    global BEV_OBJECT_TRACKS
    bev_image = np.full((BEV_HEIGHT, BEV_WIDTH, 3), BEV_BACKGROUND_COLOR, dtype=np.uint8)
    ego_center_x, ego_center_y = VEHICLE_POSITION_BEV

    # Draw subtle grid (as before)
    for i in range(1, 6):
        line_y = ego_center_y - int( (BEV_HEIGHT * 0.9 / 5) * i )
        if line_y < 0: break
        cv2.line(bev_image, (0, line_y), (BEV_WIDTH, line_y), (60,60,60), 1)
    # ... (rest of grid drawing) ...

    cv2.rectangle(bev_image, (ego_center_x - EGO_VEHICLE_RECT_W//2, ego_center_y - EGO_VEHICLE_RECT_H//2),
                  (ego_center_x + EGO_VEHICLE_RECT_W//2, ego_center_y + EGO_VEHICLE_RECT_H//2), EGO_VEHICLE_COLOR_BEV, -1)
    cv2.line(bev_image, (ego_center_x, ego_center_y - EGO_VEHICLE_RECT_H//2), (ego_center_x, ego_center_y - EGO_VEHICLE_RECT_H//2 - 6), EGO_VEHICLE_COLOR_BEV, 3)

    current_detections_for_tracking = []
    if not (yolo_results and len(yolo_results) > 0 and yolo_results[0].boxes and raw_depth_map_for_color is not None):
        for track_id, trail in list(BEV_OBJECT_TRACKS.items()):
            for i, (tx, ty, tcolor) in enumerate(trail):
                alpha_trail = 1.0 - ( (len(trail) - 1 - i) / MAX_TRAIL_LENGTH )
                trail_dot_color = (int(tcolor[0]*alpha_trail), int(tcolor[1]*alpha_trail), int(tcolor[2]*alpha_trail))
                # Use BEV_DOT_RADIUS_MIN defined at the top of this cell
                cv2.circle(bev_image, (tx, ty), max(1, BEV_DOT_RADIUS_MIN -1 if BEV_DOT_RADIUS_MIN > 1 else 1), trail_dot_color, -1)
        # ... (Legend drawing) ...
        return bev_image

    boxes_data = yolo_results[0].boxes
    for i in range(len(boxes_data.xyxy)):
        box=boxes_data.xyxy[i].cpu().numpy(); conf=boxes_data.conf[i].cpu().numpy()
        cls_id=int(boxes_data.cls[i].cpu().numpy()); class_name=yolo_model_names[cls_id]
        if ENHANCED_TARGET_CLASSES and class_name not in ENHANCED_TARGET_CLASSES: continue
        if conf < ENHANCED_CONFIDENCE_THRESHOLD: continue
        x1_obj, _, x2_obj, _ = map(int, box); center_x_orig_obj = (x1_obj + x2_obj) // 2
        object_raw_depth_value = get_raw_depth_at_center_for_enhancements(raw_depth_map_for_color, box, class_name)

        dot_color = PDF_COLOR_FAR # Uses PDF_COLOR_FAR from this cell's top
        dot_radius = BEV_DOT_RADIUS_FAR # Uses BEV_DOT_RADIUS_FAR from this cell's top
        bev_y_fraction_from_ego = 1.0

        if object_raw_depth_value is not None and (current_frame_max_raw > current_frame_min_raw):
            depth_range_frame = current_frame_max_raw - current_frame_min_raw
            close_thresh_val = current_frame_min_raw + depth_range_frame * PDF_RAW_VAL_CLOSE_THRESHOLD
            medium_thresh_val = current_frame_min_raw + depth_range_frame * PDF_RAW_VAL_MEDIUM_THRESHOLD
            if object_raw_depth_value >= close_thresh_val:
                dot_color = PDF_COLOR_CLOSE; dot_radius = BEV_DOT_RADIUS_DANGER
            elif object_raw_depth_value >= medium_thresh_val:
                dot_color = PDF_COLOR_MEDIUM; dot_radius = BEV_DOT_RADIUS_CAUTION
            clamped_depth = np.clip(object_raw_depth_value, current_frame_min_raw, current_frame_max_raw)
            if depth_range_frame > 1e-5: bev_y_fraction_from_ego = (current_frame_max_raw - clamped_depth) / depth_range_frame
            else: bev_y_fraction_from_ego = 0.0

        y_base = ego_center_y - int(bev_y_fraction_from_ego * BEV_Y_SCALE_RAW_FRACTION)
        perspective_x_factor = 1.0 - (0.4 * bev_y_fraction_from_ego)
        x_offset_from_center = ((center_x_orig_obj / original_frame_width) - 0.5) * BEV_X_SCALE_FACTOR * perspective_x_factor
        x_base = ego_center_x + int(x_offset_from_center)
        x_base = np.clip(x_base,0,BEV_WIDTH-1); y_base = np.clip(y_base, BEV_HORIZON_Y if 'BEV_HORIZON_Y' in globals() else 0, BEV_HEIGHT-1) # Use BEV_HORIZON_Y if defined

        current_detections_for_tracking.append({'x_base': x_base, 'y_base': y_base,
                                                'color': dot_color, 'radius': dot_radius,
                                                'raw_depth': object_raw_depth_value, 'class_name': class_name})

    update_bev_tracks(current_detections_for_tracking)

    for track_id, trail in list(BEV_OBJECT_TRACKS.items()):
        for i_trail, (tx_base, ty_base, tcolor) in enumerate(trail):
            if i_trail < len(trail) - 1:
                alpha_trail = 0.1 + 0.9 * (i_trail / MAX_TRAIL_LENGTH)
                trail_dot_color = (int(tcolor[0]*alpha_trail), int(tcolor[1]*alpha_trail), int(tcolor[2]*alpha_trail))
                # Use BEV_DOT_RADIUS_MIN defined at the top of this cell
                trail_radius = BEV_DOT_RADIUS_MIN - 2 + int((i_trail / MAX_TRAIL_LENGTH) * 2) # Ensure trail_radius >= 1
                cv2.circle(bev_image, (tx_base, ty_base), max(1, trail_radius), trail_dot_color, -1)

        current_x_base, current_y_base, current_color = trail[-1]
        current_radius = BEV_DOT_RADIUS_FAR # Uses BEV_DOT_RADIUS_* from this cell
        if current_color == PDF_COLOR_CLOSE: current_radius = BEV_DOT_RADIUS_DANGER
        elif current_color == PDF_COLOR_MEDIUM: current_radius = BEV_DOT_RADIUS_CAUTION

        dot_is_visible = True
        if current_color == PDF_COLOR_CLOSE: # Uses PDF_COLOR_CLOSE from this cell
            if (current_frame_number // (BLINK_FRAME_INTERVAL // 2)) % 2 == 0: dot_is_visible = False # Uses BLINK_FRAME_INTERVAL

        if dot_is_visible:
            cv2.circle(bev_image, (current_x_base, current_y_base), current_radius, current_color, -1)
            cv2.circle(bev_image, (current_x_base, current_y_base), current_radius, BEV_DOT_BORDER_COLOR, BEV_DOT_BORDER_THICKNESS)
            object_bev_height_px = BEV_OBJECT_DEFAULT_HEIGHT_PX if 'BEV_OBJECT_DEFAULT_HEIGHT_PX' in globals() else 4 # Use if defined
            display_height_px = object_bev_height_px
            if current_color == PDF_COLOR_CLOSE: display_height_px = int(object_bev_height_px * 1.5)
            elif current_color == PDF_COLOR_MEDIUM: display_height_px = int(object_bev_height_px * 1.2)
            y_top = max(BEV_HORIZON_Y if 'BEV_HORIZON_Y' in globals() else 0, current_y_base - display_height_px)
            cv2.line(bev_image, (current_x_base, current_y_base), (current_x_base, y_top), current_color, max(1, current_radius // 2))

    # Legend uses PDF_COLOR_* from this cell
    cv2.putText(bev_image,"BEV Zones:",(5,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,COLOR_TEXT_BOX,1)
    cv2.circle(bev_image,(10,30),5,PDF_COLOR_CLOSE,-1); cv2.line(bev_image,(10,30),(10,30),BEV_DOT_BORDER_COLOR,1); cv2.putText(bev_image,"Close",(20,35),cv2.FONT_HERSHEY_SIMPLEX,0.4,COLOR_TEXT_BOX,1)
    cv2.circle(bev_image,(10,50),5,PDF_COLOR_MEDIUM,-1); cv2.line(bev_image,(10,50),(10,50),BEV_DOT_BORDER_COLOR,1); cv2.putText(bev_image,"Medium",(20,55),cv2.FONT_HERSHEY_SIMPLEX,0.4,COLOR_TEXT_BOX,1)
    cv2.circle(bev_image,(10,70),5,PDF_COLOR_FAR,-1); cv2.line(bev_image,(10,70),(10,70),BEV_DOT_BORDER_COLOR,1); cv2.putText(bev_image,"Far",(20,75),cv2.FONT_HERSHEY_SIMPLEX,0.4,COLOR_TEXT_BOX,1)
    return bev_image

print("✅ Cell 5: BEV with blinking/sized dots & trails (constants self-contained & checked).")
print("-" * 70)

✅ Cell 5: BEV with blinking/sized dots & trails (constants self-contained & checked).
----------------------------------------------------------------------


In [31]:
#@title 6. WebSocket Server Definitions (BEV with Blinking/Sized Dots & Trails)
import asyncio
import websockets
import nest_asyncio
from pyngrok import ngrok
import time
import os
import cv2
import numpy as np
import traceback
import torch
from ultralytics import YOLO
import logging
from google.colab import files # For download option in Colab
# from collections import deque # deque is used in Cell 5's update_bev_tracks

# --- Logging Configuration ---
LOGGING_LEVEL = logging.INFO # Change to DEBUG for very verbose logs
logging.basicConfig(
    level=LOGGING_LEVEL,
    format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
try:
    nest_asyncio.apply()
    logging.info("nest_asyncio applied.")
except RuntimeError:
    logging.warning("nest_asyncio already applied or cannot be applied.")

# --- WebSocket and PDF Line Drawing Constants ---
WEBSOCKET_PORT = 8765
active_connections = set()
PDF_CONFIDENCE_THRESHOLD = 0.4
PDF_TARGET_CLASSES = [] # From your PDF, not used by enhancements unless ENHANCED_TARGET_CLASSES is set
PDF_LINE_THICKNESS = 2
PDF_RAW_VAL_CLOSE_THRESHOLD = 0.6
PDF_RAW_VAL_MEDIUM_THRESHOLD = 0.3
PDF_COLOR_CLOSE = (0, 0, 255)    # Red
PDF_COLOR_MEDIUM = (0, 255, 255) # Yellow
PDF_COLOR_FAR = (0, 255, 0)      # Green

# --- Model Input Size for Depth Model (from PDF Cell 15) ---
model_input_size = 518

# --- Video Output Configuration ---
OUTPUT_VIDEO_FILENAME = "processed_video_output.mp4"
ENABLE_VIDEO_WRITING = True # Set to False to disable writing video to file
output_video_writer = None # Global VideoWriter object

# --- Helper: Get Depth at Point (for PDF Line Logic - from PDF Cell 12) ---
def get_depth_at_point_pdf(raw_depth_map_input, x, y):
    if raw_depth_map_input is None: return None
    try:
        h, w = raw_depth_map_input.shape[:2]
        safe_y = max(0, min(h - 1, int(round(y))))
        safe_x = max(0, min(w - 1, int(round(x))))
        val = raw_depth_map_input[safe_y, safe_x]
        return val if np.isfinite(val) else None
    except Exception as e:
        logging.error(f"Err PDF depth @({x},{y}): {e}", exc_info=False)
        return None

# --- Helper: Draw Colored Depth Line (for PDF Line Logic - from PDF Cell 12-13) ---
def draw_colored_relative_depth_line_pdf(image, obj_center_x, obj_center_y, object_raw_depth_value, frame_min_raw, frame_max_raw):
    if object_raw_depth_value is None or not (frame_max_raw > frame_min_raw) : return
    h, w = image.shape[:2]; cam_ox, cam_oy = w // 2, h - 1
    depth_range = frame_max_raw - frame_min_raw
    close_thresh = frame_min_raw + depth_range * PDF_RAW_VAL_CLOSE_THRESHOLD
    medium_thresh = frame_min_raw + depth_range * PDF_RAW_VAL_MEDIUM_THRESHOLD
    line_color = PDF_COLOR_FAR
    if object_raw_depth_value >= close_thresh: line_color = PDF_COLOR_CLOSE
    elif object_raw_depth_value >= medium_thresh: line_color = PDF_COLOR_MEDIUM
    try:
        cv2.line(image, (cam_ox, cam_oy), (int(round(obj_center_x)), int(round(obj_center_y))), line_color, PDF_LINE_THICKNESS)
    except Exception as e:
        logging.error(f"[PDF Line Err] Draw: {e}", exc_info=False)


async def video_stream_server(websocket):
    global _depth_model, _yolo_model, DEVICE, _confirmed_video_path, active_connections
    global MODEL_TYPE, ENHANCED_CONFIDENCE_THRESHOLD # ENHANCED_CONFIDENCE_THRESHOLD is from Cell 5
    global output_video_writer, ENABLE_VIDEO_WRITING, OUTPUT_VIDEO_FILENAME
    # PREVIOUS_DEPTH_MAP_SMOOTH is also global if used by Cell 5's BEV or other logic if re-enabled
    # For this version, direct raw_depth_map is used for BEV coloring consistency.

    connection_id = websocket.remote_address if websocket.remote_address else f"UnknownClient-{time.time():.0f}"
    active_connections.add(websocket)
    logging.info(f"Client connected: {connection_id}. Total: {len(active_connections)}")

    if not all([_depth_model, _yolo_model, _confirmed_video_path, MODEL_TYPE]):
        reason = "Server error: Missing critical component(s)."
        logging.error(f"[{connection_id}] Pre-check failed: {reason}")
        await websocket.close(code=1011, reason=reason.strip()); active_connections.discard(websocket); return

    video_path_to_use = _confirmed_video_path
    cap = cv2.VideoCapture(video_path_to_use)
    if not cap.isOpened():
        logging.error(f"[{connection_id}] Error: Cannot open video: {video_path_to_use}")
        await websocket.close(code=1011, reason="Server error: Cannot open video file."); active_connections.discard(websocket); return

    fps = cap.get(cv2.CAP_PROP_FPS); w_vid = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h_vid = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    logging.info(f"[{connection_id}] Vid: {w_vid}x{h_vid} @ {fps:.1f}FPS. DepthM: {MODEL_TYPE}, Enh.YOLO Conf: {ENHANCED_CONFIDENCE_THRESHOLD}, PDF.YOLO Conf: {PDF_CONFIDENCE_THRESHOLD}")

    if ENABLE_VIDEO_WRITING and output_video_writer is None:
        preview_h_vid = max(120, h_vid // 3); combined_h = h_vid + preview_h_vid
        fourcc = cv2.VideoWriter_fourcc(*'mp4v'); out_path = os.path.join("/content/", OUTPUT_VIDEO_FILENAME)
        try:
            output_video_writer = cv2.VideoWriter(out_path, fourcc, fps if fps > 0 else 10, (w_vid, combined_h))
            if output_video_writer.isOpened(): logging.info(f"Output video: {out_path} @ {w_vid}x{combined_h}")
            else: logging.error(f"Failed to open VideoWriter for {out_path}"); output_video_writer = None
        except Exception as e_vw: logging.error(f"VideoWriter Exception: {e_vw}"); output_video_writer = None

    frame_count = 0; timers = {k:0 for k in ["depth","yolo","draw_pdf","draw_enh","encode","send","loop_proc"]}
    start_process_time = time.time()

    try:
        while cap.isOpened():
            loop_s = time.time(); ret, frame = cap.read(); frame_count += 1
            if not ret or frame is None: logging.info(f"End of video or bad frame {frame_count}."); break
            output_frame = frame.copy()

            depth_s = time.time(); depth_map_raw = None
            try:
                with torch.no_grad(): depth_map_raw = _depth_model.infer_image(frame, input_size=model_input_size)
                if depth_map_raw is None: logging.error(f"!!!! Fr{frame_count}: _depth_model.infer_image returned None !!!!")
                else: logging.debug(f"Fr{frame_count}: Raw depth OK. Shape:{depth_map_raw.shape}")
            except Exception as e: logging.error(f"!!!! Fr{frame_count}: EXCEPTION depth inference: {e} !!!!", exc_info=LOGGING_LEVEL <= logging.DEBUG); depth_map_raw = None
            timers["depth"] += time.time() - depth_s

            yolo_s = time.time(); yolo_results = None
            try: yolo_results = _yolo_model(frame, device=DEVICE, verbose=False, conf=min(PDF_CONFIDENCE_THRESHOLD, ENHANCED_CONFIDENCE_THRESHOLD))
            except Exception as e: logging.error(f"Fr{frame_count}: YOLO Err: {e}", exc_info=LOGGING_LEVEL <= logging.DEBUG)
            timers["yolo"] += time.time() - yolo_s

            current_frame_min_raw, current_frame_max_raw = 0, 1
            if depth_map_raw is not None:
                finite_map = depth_map_raw[np.isfinite(depth_map_raw)]
                if finite_map.size > 0:
                    current_frame_min_raw = np.min(finite_map)
                    current_frame_max_raw = np.max(finite_map)

            pdf_draw_s = time.time()
            if depth_map_raw is not None and (current_frame_max_raw > current_frame_min_raw) and yolo_results and len(yolo_results) > 0 and yolo_results[0].boxes is not None:
                 boxes_pdf = yolo_results[0].boxes
                 for i in range(len(boxes_pdf.xyxy)):
                    if boxes_pdf.conf[i] < PDF_CONFIDENCE_THRESHOLD: continue
                    x1p,y1p,x2p,y2p=map(int,boxes_pdf.xyxy[i].cpu().numpy()); cxp,cyp=(x1p+x2p)/2,(y1p+y2p)/2
                    depth_val_line = get_depth_at_point_pdf(depth_map_raw, cxp, cyp)
                    draw_colored_relative_depth_line_pdf(output_frame, cxp, cyp, depth_val_line, current_frame_min_raw, current_frame_max_raw)
            timers["draw_pdf"] += time.time() - pdf_draw_s

            enh_draw_s = time.time()
            output_frame_with_enhancements = draw_ar_bounding_boxes(output_frame, yolo_results,
                                                                    depth_map_raw, current_frame_min_raw, current_frame_max_raw,
                                                                    _yolo_model.names)
            # Pass frame_count to create_enhanced_bev for blinking logic
            bev_display = create_enhanced_bev(yolo_results,
                                              depth_map_raw, current_frame_min_raw, current_frame_max_raw,
                                              _yolo_model.names, w_vid, frame_count) # Added frame_count
            timers["draw_enh"] += time.time() - enh_draw_s

            depth_colored_display = colorize_relative_depth(depth_map_raw)
            if depth_colored_display is None:
                depth_colored_display = np.zeros((h_vid, w_vid, 3), dtype=np.uint8)
                cv2.putText(depth_colored_display, "Depth Fail" if depth_map_raw is None else "Colorize Fail", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
            elif depth_colored_display.shape[:2] != (h_vid,w_vid): depth_colored_display = cv2.resize(depth_colored_display, (w_vid,h_vid))

            preview_h = max(120, h_vid // 3); preview_w_half = w_vid // 2
            bev_preview = cv2.resize(bev_display, (preview_w_half, preview_h), interpolation=cv2.INTER_AREA)
            depth_preview = cv2.resize(depth_colored_display, (preview_w_half, preview_h), interpolation=cv2.INTER_AREA)
            aux_row = np.hstack((bev_preview, depth_preview))
            if aux_row.shape[1] != w_vid: aux_row = cv2.resize(aux_row, (w_vid, preview_h), interpolation=cv2.INTER_AREA)
            final_display = np.vstack((output_frame_with_enhancements, aux_row))

            if ENABLE_VIDEO_WRITING and output_video_writer is not None and output_video_writer.isOpened():
                writer_expected_h = h_vid + preview_h
                if final_display.shape[0] != writer_expected_h or final_display.shape[1] != w_vid:
                    final_display_for_write = cv2.resize(final_display, (w_vid, writer_expected_h))
                    output_video_writer.write(final_display_for_write)
                else: output_video_writer.write(final_display)

            encode_s = time.time(); ret_enc, buffer = cv2.imencode('.jpg', final_display, [int(cv2.IMWRITE_JPEG_QUALITY), 75])
            timers["encode"] += time.time() - encode_s;
            if not ret_enc: logging.warning(f"JPEG encode fail fr {frame_count}"); continue
            send_s = time.time(); await websocket.send(buffer.tobytes()); timers["send"] += time.time() - send_s
            timers["loop_proc"] += time.time() - loop_s
            await asyncio.sleep(0.001)

    except websockets.exceptions.ConnectionClosed: logging.info(f"[{connection_id}] Client disconnected.")
    except websockets.exceptions.ConnectionClosedOK: logging.info(f"[{connection_id}] Client disconnected normally.")
    except Exception as e: logging.error(f"!!! UNEXPECTED ERROR for {connection_id}: {e} !!!", exc_info=True); traceback.print_exc()
    finally:
        duration = time.time() - start_process_time; cap.release(); active_connections.discard(websocket)
        # PREVIOUS_DEPTH_MAP_SMOOTH is defined in Cell 5, ensure it's handled if used
        if 'PREVIOUS_DEPTH_MAP_SMOOTH' in globals() and not active_connections: globals()['PREVIOUS_DEPTH_MAP_SMOOTH'] = None

        if not active_connections and output_video_writer is not None:
            if output_video_writer.isOpened(): logging.info(f"Finalizing video: {OUTPUT_VIDEO_FILENAME}"); output_video_writer.release()
            output_video_writer = None
            if 'google.colab' in sys.modules:
                try:
                    out_path_final = os.path.join("/content/", OUTPUT_VIDEO_FILENAME)
                    if os.path.exists(out_path_final): print(f"Offering download: {out_path_final}"); files.download(out_path_final)
                except Exception as e_dl: print(f"Download err: {e_dl}")
        logging.info(f"[{connection_id}] Closed. Processed {frame_count} frames in {duration:.2f}s.")
        if frame_count > 0:
             log_msg = f"AvgTimes(ms/fr) for {connection_id}: "; timers_list = [(k,v) for k,v in timers.items()]; log_msg += " | ".join([f"{k}={(v/frame_count)*1000:.1f}" for k,v in timers_list]); logging.info(log_msg)
        logging.info(f"Remaining clients: {len(active_connections)}")

async def start_server():
    global WEBSOCKET_PORT, NGROK_AUTH_TOKEN, active_connections, _depth_model, _yolo_model, _confirmed_video_path, MODEL_TYPE, output_video_writer
    http_tunnel=None; video_path_exists = False
    if _confirmed_video_path and isinstance(_confirmed_video_path, str): video_path_exists = os.path.exists(_confirmed_video_path)
    if not all([_depth_model, _yolo_model, _confirmed_video_path, MODEL_TYPE, video_path_exists]): logging.error("!!! ABORT: Critical components not ready for server start."); return
    ws_url = f"ws://localhost:{WEBSOCKET_PORT}"
    # Ensure NGROK_AUTH_TOKEN is checked against your actual placeholder string from Cell 1 (e.g. "YOUR_ACTUAL_NGROK_AUTH_TOKEN_HERE")
    if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_ACTUAL_NGROK_AUTH_TOKEN_HERE" and NGROK_AUTH_TOKEN.strip() != "":
        try:
            for t in ngrok.get_tunnels():
                if str(t.config.get('addr','')).endswith(str(WEBSOCKET_PORT)): ngrok.disconnect(t.public_url); await asyncio.sleep(1)
            http_tunnel=ngrok.connect(WEBSOCKET_PORT,"http",bind_tls=True); p_url=http_tunnel.public_url
            ws_url = p_url.replace("https://","wss://") if p_url.startswith("https") else p_url.replace("http://","ws://")
            logging.info(f"Ngrok tunnel: {p_url} (HTTP), {ws_url} (WS/WSS)")
        except Exception as e: logging.error(f"NGROK FAIL: {e}", exc_info=True); logging.warning("Ngrok failed. Local only.")
    else: logging.warning("Ngrok Auth Token not set/placeholder. Skipping ngrok.")
    print(f"\n{'='*70}\n=== WS Server Ready ===\n=== Connect client to: {ws_url} ===\n{'='*70}\n")
    server_inst=None
    try:
        server_inst = await websockets.serve(video_stream_server, "0.0.0.0", WEBSOCKET_PORT, ping_interval=20, ping_timeout=20, max_size=15*1024*1024)
        logging.info(f"WebSocket server on 0.0.0.0:{WEBSOCKET_PORT}. Waiting for connections..."); await asyncio.Future()
    except Exception as e: logging.error(f"WS SERVER Error on startup: {e}", exc_info=True)
    finally:
        logging.info("Initiating server shutdown sequence...")
        if server_inst: server_inst.close(); await server_inst.wait_closed(); logging.info("WS server stopped.")
        if output_video_writer is not None:
            if output_video_writer.isOpened(): logging.info("Releasing video writer (server stop)..."); output_video_writer.release()
            output_video_writer = None
            if 'google.colab' in sys.modules:
                out_path_final = os.path.join("/content/", OUTPUT_VIDEO_FILENAME)
                if os.path.exists(out_path_final): print(f"Offering download (server stop): {out_path_final}"); files.download(out_path_final)
        if http_tunnel:
          try: ngrok.disconnect(http_tunnel.public_url); ngrok.kill(); logging.info("Ngrok stopped.")
          except Exception as e_ngrok: logging.warning(f"Ngrok disconnect error: {e_ngrok}")
        active_connections.clear(); logging.info("Server shutdown complete.")

def run_websocket_server():
    global _confirmed_video_path, _depth_model, _yolo_model, MODEL_TYPE
    global BEV_OBJECT_TRACKS, NEXT_TRACK_ID # For resetting tracks

    # Reset BEV tracks each time server runs
    BEV_OBJECT_TRACKS = {}
    NEXT_TRACK_ID = 0

    # Removed preload_icons as we are using dots
    logging.info("BEV will use colored dots (icons are not preloaded in this version).")

    if 'input_video_path' in globals() and globals()['input_video_path']: _confirmed_video_path = globals()['input_video_path']
    elif '_confirmed_video_path' not in globals() or not globals()['_confirmed_video_path']: _confirmed_video_path = None

    path_ok = False;
    if _confirmed_video_path and isinstance(_confirmed_video_path, str): path_ok = os.path.exists(_confirmed_video_path)
    model_type_ok = 'MODEL_TYPE' in globals() and globals()['MODEL_TYPE'] is not None
    depth_ok = '_depth_model' in globals() and globals()['_depth_model'] is not None
    yolo_ok = '_yolo_model' in globals() and globals()['_yolo_model'] is not None

    if path_ok and depth_ok and yolo_ok and model_type_ok:
        logging.info("All prerequisites met. Starting server event loop...");
        asyncio.run(start_server())
    else:
        logging.error("Prerequisites not met. Server startup aborted.")
        if not path_ok: logging.error(f"  - Video path issue. Current _confirmed_video_path: '{_confirmed_video_path}'.")
        if not depth_ok: logging.error("  - Depth model issue (_depth_model is None).")
        if not yolo_ok: logging.error("  - YOLO model issue (_yolo_model is None).")
        if not model_type_ok: logging.error("  - MODEL_TYPE not defined.")

print("✅ Cell 6: WebSocket Server (BEV with Blinking/Sized Dots & Trails).")
print("   Set LOGGING_LEVEL. ENABLE_VIDEO_WRITING to save.")
print("   Run 'run_websocket_server()' in Cell 7 to start.")
print("-" * 70)

✅ Cell 6: WebSocket Server (BEV with Blinking/Sized Dots & Trails).
   Set LOGGING_LEVEL. ENABLE_VIDEO_WRITING to save.
   Run 'run_websocket_server()' in Cell 7 to start.
----------------------------------------------------------------------


In [32]:
#@title 7. Start the Streaming Server
import os
import torch # For torch.cuda.is_available() and empty_cache()

print("--- Initiating Server Start Sequence (Cell 7) ---")

# --- Prerequisite Checks (Final verification before calling run_websocket_server) ---
# These check the global variables that should have been set by earlier cells.
prerequisites_met = True

# Check 1: Depth model (should be loaded in Cell 2 and globally accessible as _depth_model)
if '_depth_model' not in globals() or globals()['_depth_model'] is None:
    print("❌ ERROR (Cell 7): Depth model (_depth_model) not loaded or not in global scope.")
    prerequisites_met = False
else:
    print("✅ Prerequisite check: Depth model found.")

# Check 2: YOLO model (should be loaded in Cell 2 and globally accessible as _yolo_model)
if '_yolo_model' not in globals() or globals()['_yolo_model'] is None:
    print("❌ ERROR (Cell 7): YOLO model (_yolo_model) not loaded or not in global scope.")
    prerequisites_met = False
else:
    print("✅ Prerequisite check: YOLO model found.")

# Check 3: Video path (should be set by Cell 4 as input_video_path, then used by run_websocket_server)
# run_websocket_server itself will check _confirmed_video_path
# Here we check the input_video_path that run_websocket_server will use
temp_confirmed_video_path = None
if 'input_video_path' in globals() and globals()['input_video_path'] is not None:
    temp_confirmed_video_path = globals()['input_video_path']

if temp_confirmed_video_path is None:
    print("❌ ERROR (Cell 7): Video path (input_video_path) not set. Run Cell 4.")
    prerequisites_met = False
elif not os.path.exists(temp_confirmed_video_path):
    print(f"❌ ERROR (Cell 7): Video file missing: '{temp_confirmed_video_path}'. Run Cell 4.")
    prerequisites_met = False
else:
    print(f"✅ Prerequisite check: Video file found at '{temp_confirmed_video_path}'.")

# Check 4: Ngrok Token (from Cell 1, used by start_server called by run_websocket_server)
if 'NGROK_AUTH_TOKEN' not in globals() or \
   globals()['NGROK_AUTH_TOKEN'] == "YOUR_NGROK_AUTH_TOKEN" or \
   not globals()['NGROK_AUTH_TOKEN']:
    print("⚠️ WARNING (Cell 7): Ngrok Auth Token (NGROK_AUTH_TOKEN) missing or placeholder. Ngrok tunneling might fail or be skipped.")
    # Not setting prerequisites_met to False, as local server can still run.
else:
    print("✅ Prerequisite check: Ngrok token seems set.")

# --- Execute Server Start ---
if prerequisites_met:
    print("\nAll critical prerequisites met. Calling 'run_websocket_server()'...")
    print("Server will now start. Execution will block here until stopped (e.g., by KeyboardInterrupt).")
    print("Look for the 'wss://...' URL (if using Ngrok) or 'ws://localhost...' URL in the logs above to connect your client.")
    print("-" * 70 + "\n")

    # Clear CUDA cache if GPU is used, before starting the blocking server call
    if DEVICE == 'cuda' and torch.cuda.is_available():
        print("Attempting to clear CUDA cache before starting server...")
        try:
            torch.cuda.empty_cache()
            print("CUDA cache cleared.")
        except Exception as e_cache:
            print(f"Warning: Error clearing CUDA cache: {e_cache}")

    # Call the main function defined in Cell 6
    run_websocket_server() # This is a blocking call

    # --- Code below runs only AFTER server stops ---
    print("\n" + "-" * 70)
    print("--- Server execution finished or interrupted (Cell 7) ---")
else:
    print("\n--- ❌ Server Start Aborted due to missing prerequisites (Cell 7) ---")
    print("--- Please check the error messages above and ensure all setup cells (1-4) have run successfully. ---")

print("-" * 70)

--- Initiating Server Start Sequence (Cell 7) ---
✅ Prerequisite check: Depth model found.
✅ Prerequisite check: YOLO model found.
✅ Prerequisite check: Video file found at '/content/Generate_Crowded_Street_Scene_Video.mp4'.
✅ Prerequisite check: Ngrok token seems set.

All critical prerequisites met. Calling 'run_websocket_server()'...
Server will now start. Execution will block here until stopped (e.g., by KeyboardInterrupt).
Look for the 'wss://...' URL (if using Ngrok) or 'ws://localhost...' URL in the logs above to connect your client.
----------------------------------------------------------------------

Attempting to clear CUDA cache before starting server...
CUDA cache cleared.

=== WS Server Ready ===
=== Connect client to: wss://aff947d81a8a.ngrok-free.app ===

Offering download: /content/processed_video_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Offering download: /content/processed_video_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 